In [ ]:
import sys
import math
from math import log, sqrt, exp
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
from PyQt5.QtWidgets import QMainWindow, QApplication, QMessageBox
from PyQt5 import uic
from scipy.stats import norm
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

class OptionCalculator:
    
    """Black Scholes"""
    def blackscholes(S,K,r,q,T,sigma,N=100):
        d1 = (log(S/K)+(r-q+sigma**2/2)*T)/(sigma*sqrt(T))
        d2 = d1-sigma*sqrt (T)
        
        call_blackscholes = S*exp(-q*T)*norm.cdf(d1)-K*exp(-r*T)*norm.cdf(d2)
        put_blackscholes = K*exp(-r*T)*norm.cdf(-d2)-S*exp(-q*T)*norm.cdf(-d1)

        return (round(call_blackscholes,2), round(put_blackscholes,2))
    
    """Binomial Tree"""
    def binom(S,K,r,q,T,sigma,N=100):
        dT=T/N
        u=math.exp(sigma*math.sqrt(dT))
        d=1/u
        p=(math.exp((r-q)*dT)-d)/(u-d)
        
        fc=[0]*(N+1)
        fp=[0]*(N+1)
        fc[N]=[max(0, S*(u**j)*(d**(N-j))-K) for j in range(N+1)]
        fp[N]=[max(0, K-S*(u**j)*(d**(N-j))) for j in range(N+1)]
        
        p1=1-p
        ert=math.exp(-r*dT)
        for i in range(N-1,0-1,-1):
            # nested list with list comprehension
            fc[i]=[ert*(p*fc[i+1][j+1]+p1*fc[i+1][j]) for j in range(i+1)]
            fp[i]=[ert*(p*fp[i+1][j+1]+p1*fp[i+1][j]) for j in range(i+1)]
        
        call_binom = fc[0][0]
        put_binom = fp[0][0]
        return (round(call_binom,2), round(put_binom,2))

    """Arbitrary values to calculate option prices for both Black Scholes and binomial Tree"""
    #if __name__=='__main__':
    #    S=50.0; K=50.0; T=183/365 
    #    sigma=0.4; r=0.04; q=0.01
    #    print('Blackscholes Call: {0[0]}, Blackscholes Put: {0[1]}'.format(
    #         blackscholes(S,K,r,q,T,sigma)))
    #    print('Binomial Call: {0[0]}, Binomial Put: {0[1]}'.format(
    #                binom(S,K,r,q,T,sigma)))

"""Main GUI Class"""
qtCreatorFile = "black_scholes_calc.ui" 
Ui_MainWindow, QtBaseClass = uic.loadUiType(qtCreatorFile)

class Main(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super().__init__()
        self.setupUi(self) 
        self.pushButton_Calculate.clicked.connect(self.Calculate)
        self.pushButton_Reset.clicked.connect(self.reset_fields)
        self.option_calculator = OptionCalculator()
        self.chart_widget = None

    """Please edit this portion accordingly"""
    def Calculate(self):
        inputs = {
            'S': float(self.lineEdit_SpotPrice.text()),
            'K': float(self.lineEdit_StrikePrice.text()),
            'T': float(self.lineEdit_TimetoExpiry.text()),
            'r': float(self.lineEdit_InterestRate.text()),
            'q': float(self.lineEdit_Dividend.text()),
            'sigma': float(self.lineEdit_Volatility.text()),
        }
        
        call_blackscholes, put_blackscholes = self.option_calculator.blackscholes(inputs)
        call_binom, put_binom = self.option_calculator.binom(inputs)
        
        self.lineEdit_BSCallPrice.setText(str(call_blackscholes))
        self.lineEdit_BSPutPrice.setText(str(put_blackscholes))
        self.lineEdit_BNCallPrice.setText(str(call_binom))
        self.lineEdit_BNPutPrice.setText(str(put_binom))

    def reset_fields(self):
        self.lineEdit_SpotPrice.clear()
        self.lineEdit_StrikePrice.clear()
        self.lineEdit_TimetoExpiry.clear()
        self.lineEdit_InterestRate.clear()
        self.lineEdit_Volatility.clear()
        self.lineEdit_Dividend.clear()
        self.lineEdit_CallPrice.clear()
        self.lineEdit_PutPrice.clear()
        
    def show_error(self, title, message):
        error_dialog = QMessageBox()
        error_dialog.setIcon(QMessageBox.Critical)
        error_dialog.setWindowTitle(title)
        error_dialog.showMessage(message)
        error_dialog.exec_()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    main = Main()
    main.show()
    sys.exit(app.exec_())